#### LeetCode - SQL - #180

Refer [180. Consecutive Numbers](https://leetcode.com/problems/consecutive-numbers/description/)

Find all numbers that appear at least three times consecutively.

Return the result table in  **any order**.

In [ ]:
data_numbers = [[1, 1], [2, 1], [3, 1], [4, 2], [5, 1], [6, 2], [7, 2]]
columns_numbers = ['id', 'num']
schema_numbers = {'id':'Int64', 'num':'Int64'}

In [ ]:
import pandas

pandas_df_numbers = pandas.DataFrame(data=data_numbers, columns=columns_numbers).astype(schema_numbers)

In [ ]:
from pyspark.sql import SparkSession

spark_context = SparkSession.builder.appName("LeetCode SQL").getOrCreate()

# Spark DataFrame using Pandas DataFrame
df_numbers = spark_context.createDataFrame(pandas_df_numbers)
df_numbers.printSchema()
df_numbers.show(5)

#### Using Window Functions

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lag, lead

window_spec = Window.orderBy(df_numbers.id)
df_numbers = \
    df_numbers \
        .withColumn("lag_num", lag(df_numbers.num, 1).over(window_spec)) \
        .withColumn("lead_num", lead(df_numbers.num, 1).over(window_spec))
df_numbers.show(5)

df_numbers \
    .filter((df_numbers.num == df_numbers.lag_num) & (df_numbers.num == df_numbers.lead_num)) \
    .select(df_numbers.num.alias("ConsecutiveNums")) \
    .show()

#### Using SQL Joins

In [ ]:
from pyspark.sql.functions import col

df_root = df_numbers.alias("number")
df_lag = df_numbers.alias("lag")
df_lead = df_numbers.alias("lead")

df_joined = \
    df_root.join(df_lag, (col("number.id") - 1) == col("lag.id"), "inner") \
        .join(df_lead, (col("number.id") + 1) == col("lead.id"), "inner") \
        .select(col("lag.num").alias("num_lag"), col("number.num").alias("num"), col("lead.num").alias("num_lead"))

df_joined \
    .filter((df_joined.num_lag == df_joined.num) & (df_joined.num == df_joined.num_lead)) \
    .select(df_joined.num.alias("ConsecutiveNums")) \
    .show()